<a href="https://colab.research.google.com/github/saikarthik952/Bits-Final-Project/blob/main/final_code_bits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install faker
!pip install jaxlib
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.615s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
# for command line usage
import sys, getopt


# for creating fake values
from faker import Faker
# for random sampling
from numpy.random import choice as choose
from numpy.random import seed as setseed
# for progress bar
from tqdm import tqdm
# for generating file name with current time
from datetime import datetime
# for saving the final dataframe.
import pandas as pd
from faker.providers import BaseProvider
import random
import string

class CustomProvider(BaseProvider):
    def random_12_digits(self):
        return ''.join(str(random.randint(0, 9)) for _ in range(12))
    def custom_pattern_string(self):
        uppercase_letters = ''.join(random.choice(string.ascii_uppercase) for _ in range(5))
        digits = ''.join(random.choice(string.digits) for _ in range(4))
        last_char = random.choice(string.ascii_uppercase)
        return str(uppercase_letters + digits + last_char)
    def random_txn_id(self, length=10):
        return ''.join('pay_' + random.choice(string.ascii_uppercase + string.digits) for _ in range(length))
    def custom_paragraph_with_keywords(self, keyword_list, num_sentences=1):
        sentences = []

        # Predefined sentences with keywords
        predefined_sentences = [
            "Transaction is successful for   {} Payment",
            "Customer  Identity with {}.",
            "Payment for  {}.",
            "{} value for customer",
             "A payment of {} with  was successfully processed.",
    "Your recent transaction of {} with has been confirmed.",
    "Thank you for your payment of {} with on our platform.",
    "A withdrawal of {} witj has been completed from your account.",
    "The payment of {} with was successful.",
    "Your credit card associated with {} has been charged",
    "A refund of {} with has been initiated for your recent transaction.",
    "A payment of {} with was declined. Please check your payment information.",
    "Your account has been credited with {} with for a successful transaction.",
    "{} with has been transferred to your linked bank account.",
            "This is a sample sentence that doesn't contain anything",
            "[\"IdCard\":\"{}\", \"message\":\"This is ID card\"]"
        ]

        for _ in range(num_sentences):

                # Use a predefined sentence with a keyword
              sentence_template = random.choice(predefined_sentences)
              keyword = random.choice(keyword_list)
              sentence = sentence_template.format(keyword)
              sentences.append(sentence)

        return ' '.join(sentences)





def sep_change(text, init_sep, after_sep):
    """
    A function to separate the input text into strings by the init_sep variable

    and combine the strings again by the after_sep variable.
    """
    splitted_strings = text.split(init_sep)

    combined_strings = after_sep.join(splitted_strings)

    return combined_strings

def convert_datetime_underscore(data):
    """
    A Helper function to convert all separators in datetime.now() into underscore.
    """
    now_string = str(data)

    now_string = sep_change(now_string, init_sep = "-", after_sep = "_")

    now_string = sep_change(now_string, init_sep = " ", after_sep = "_")

    now_string = sep_change(now_string, init_sep = ":", after_sep = "_")

    now_string = sep_change(now_string, init_sep = ".", after_sep = "_")

    return now_string

def _random_sep_change(data, init_sep = "-", after_sep = " ", percentage = 0.5 , seed = 7):
    """
    A function to randomly change the SSN data's separator.

    The input data is a list.
    """
    setseed(seed)
    # generate the index for replacing separator.
    replacing_indexes = choose(range(len(data)), int(len(data)*percentage))

    for each_replacing_index in replacing_indexes:
        # change the ssn data's separator from init_sep to after_sep
        data[each_replacing_index] = sep_change(data[each_replacing_index], init_sep, after_sep)

    return data





class Fake_PII():
    '''
    A class to generate a number of fake profiles, training/testing text mixed with
    different types of fake PIIs.

    Examples
    --------
    fake_ = Fake_PII()
    fake_.create_fake_profile(10)
    train_labels, train_text, train_PII = fake_.create_pii_text_train(n_text = 5)
    '''
    def __init__(self, n_profile = None,fake_profiles = None, seed = 7,\
                pii_with_text = None, pii_labels = None, PII = None):



        # initialize the Faker from faker package for fake data generation.
        try:
            self.faker = Faker()
        except ImportError as error:
            print(error.__class__.__name__ + ": " + error.message)

        self.faker.add_provider(CustomProvider)
        self.n_profile = n_profile
        self.pii_with_text = pii_with_text
        self.pii_labels = pii_labels
        self.fake_profiles = fake_profiles
        self.seed = seed
        self.PII = PII


    def create_fake_profile(self, n_profile, verbose = False, ssn_sep_change = True):

        assert isinstance(n_profile, int), "Please enter an integer\
        for the number of generated profiles."
        self.n_profile = n_profile

        fake_profiles = dict()


        fake_profiles["Aadhaar"] = [self.faker.random_12_digits() for _ in range(self.n_profile)]

        fake_profiles["Other"] = [self.faker.random_txn_id() for _ in range(self.n_profile)]



        fake_profiles["Pancard"] = [self.faker.custom_pattern_string()\
                                   for _ in range(self.n_profile)]

        self.fake_profiles = fake_profiles

        if verbose:
            return self.fake_profiles

    def _init_pii_gen_train(self):

        # generate the all possible PII implemented in the create_fake_profile methods
        self._fake_labels = list(self.fake_profiles.keys())
        # print(self._fake_labels);
        # print("1")
        # generate the None labels
        self._none_pii_labels = ["Other" for _ in range(self._n_text)]
        # print(self._none_pii_labels);
        # print("2")
        # generate the pii labels
        self.pii_labels = sorted(self._fake_labels*self._n_text)
        # print(self.pii_labels);
        # print("3")
        # generate the test with no pii
        self._fake_text_no_pii = [self.faker.custom_paragraph_with_keywords(keyword_list=['aadhaar','pancard']) for _ in range(self._n_text)]
        # print(self._fake_text_no_pii);
        # print("4")
        # mutiply the no pii text with the number of PII types
        self._init_fake_text_no_pii = self._fake_text_no_pii*len(self._fake_labels)
        # print(self._init_fake_text_no_pii);
        # print("5")

        # initialize the text mixed with PII with all "None" strings.
        self.pii_with_text = ["None" for _ in range(len(self._fake_labels)*(self._n_text))]
        # print(self.pii_with_text);
        # print("6")
        # initialize the PII with all "None" strings
        self.PII = ["None" for _ in range(self._n_text*(len(self._fake_labels)+1))]
        # print(self.PII);
        # print("7")
    def _random_pii_insert(self):
        # randomly insert PII into the text
        for index, PII in enumerate(tqdm(self.pii_labels)):
            # choose a PII value from the dictionary according to the PII type.
            #print("PII" + PII)
            PII_value = choose(self.fake_profiles[PII])
            #print("PIIval" + PII_value)
            if PII == "Other" :
               original_fake_text = self.faker.custom_paragraph_with_keywords(keyword_list=[str("TxnId" + " - " + PII_value)])
            else :
               original_fake_text = self.faker.custom_paragraph_with_keywords(keyword_list=[str(PII + " - " + PII_value)])

            # tokenized_fake_text = original_fake_text.split(" ")

            # # generate the position to fill in the PII value
            # PII_position = choose(range(len(tokenized_fake_text)+1))

            # tokenized_fake_text.insert(PII_position, PII_value)

            # one_text_mixed_with_PII = " ".join(tokenized_fake_text)

            self.pii_with_text[index] = original_fake_text
            self.PII[index] = PII_value


    def create_pii_text_train(self, n_text = 10):
        """
        A method to create the training text randomly mixed with fake PII. This
        method creates a text and mixed it with different kinds of PII, which leads
        to a total number of (num_of_PII)*(n_text) rows.

        """
        warning_text = "Please create fake profiles first with .create_fake_profile method."
        assert self.fake_profiles is not None, warning_text

        self._n_text = n_text
        # initialized a few variables for inserting PII values
        self._init_pii_gen_train()

        # randomly insert Pii text into the paragraph.
        self._random_pii_insert()


        self.pii_with_text.extend(self._fake_text_no_pii)
        self.pii_labels.extend(self._none_pii_labels)



        return self.pii_labels, self.pii_with_text, self.PII

    def _init_pii_gen_test(self):

        # generate the all possible PII implemented in the create_fake_profile methods
        self._fake_labels = list(self.fake_profiles.keys())

        # generate the None labels
        self._none_pii_labels = ["Other" for _ in range(self._n_text)]

        # generate the pii labels
        self.pii_labels = sorted(self._fake_labels)

        total_num_pii_text = (1+len(self._fake_labels))
        # generate the fake text with no pii
        self._init_fake_text_no_pii = [self.faker.custom_paragraph_with_keywords(keyword_list=['aadhaar','pancard']) for _ in range(total_num_pii_text)]

        # initialize the text mixed with PII with all "None" strings.
        self.pii_with_text = self._init_fake_text_no_pii
        # initialize the PII with all "None" strings
        self.PII = ["None" for _ in range(total_num_pii_text)]


    def create_pii_text_test(self, n_text = 10):
        """
        A method to create the testing text randomly mixed with fake PII.
        This method creates a text and mixed it with a type of PII.

        In the training text, a normal text is repeated used to insert different PIIs into
        it. In the testing text, a normal text is not intentionally repeated to insert
        different PIIs.

        """
        warning_text = "Please create fake profiles first with .create_fake_profile method."
        assert self.fake_profiles is not None, warning_text

        self._n_text = n_text

        # initialized a few variables for inserting PII values
        self._init_pii_gen_test()

        # randomly insert Pii text into the paragraph.
        self._random_pii_insert()
        # add the none labels
        #self.pii_labels.extend(self._none_pii_labels)

        return self.pii_labels, self.pii_with_text, self.PII



def write_to_disk_train(requested_train_data_size):

    fake_ = Fake_PII()
    fake_.create_fake_profile(requested_train_data_size)
    train_labels, train_text, train_PII = fake_.create_pii_text_train(n_text = requested_train_data_size)

    # save training data to disk
    train_text_with_pii = pd.DataFrame({"Text":train_text, "Labels":train_labels, "PII":train_PII})
    train_file_name = "train_text_with_pii_" + convert_datetime_underscore(datetime.now()) + ".csv"
    train_text_with_pii.to_csv(train_file_name,index=False)

def write_to_disk_test(requested_test_data_size):
    fake_ = Fake_PII()
    fake_.create_fake_profile(requested_test_data_size)
    test_labels, test_text, test_PII = fake_.create_pii_text_test(n_text = requested_test_data_size)

    # save testing data to disk
    test_text_with_pii = pd.DataFrame({"Text":test_text, "Labels":test_labels, "PII":test_PII})
    test_file_name = "test_text_with_pii_" + convert_datetime_underscore(datetime.now()) + ".csv"
    test_text_with_pii.to_csv(test_file_name,index=False)




if __name__ == "__main__":
    argument_list = sys.argv[1:]

    if "-train" in argument_list:
        train_value_index = argument_list.index("-train") + 1
        requested_train_data_size = int(argument_list[train_value_index])

        write_to_disk_train(requested_train_data_size)

    if "-test" in argument_list:
        test_value_index = argument_list.index("-test") + 1
        requested_test_data_size = int(argument_list[test_value_index])

        write_to_disk_test(requested_test_data_size)

In [ ]:
fake_ = Fake_PII()
data = fake_.create_fake_profile(100,verbose=True)
# print(data['AadhaarNumber'])
# print(data['PanCard'])
train_labels, train_text, train_PII = fake_.create_pii_text_train(n_text = 100)
for i, element in enumerate(train_labels):
  if element == 'Aadhaar' or element == 'Pancard':
    train_labels[i]= "PII"
print(train_labels)
print(train_text)
print(train_PII)

100%|██████████| 300/300 [00:00<00:00, 10107.89it/s]

['PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'PII', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'Other', 'O

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
# Function to format elapsed time
import time

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(time.strftime("%H:%M:%S", time.gmtime(elapsed_rounded)))
# Sample dataset (replace this with your actual dataset)
data = {
    'sentence': train_text,
    'label': train_labels
}

df = pd.DataFrame(data)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df['sentence'], df['label'], test_size=0.2, random_state=42)

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize inputs
def tokenize_inputs(text_list, tokenizer, max_length):
    input_ids = []
    attention_masks = []

    for text in text_list:
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = max_length,           # Pad & truncate all sentences.
                            padding='max_length',
                            truncation=True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

# Tokenize training and testing data
train_input_ids, train_attention_masks = tokenize_inputs(X_train, tokenizer, max_length=128)
test_input_ids, test_attention_masks = tokenize_inputs(X_test, tokenizer, max_length=128)

# Convert labels to torch tensors
train_labels = torch.tensor([1 if label == 'PII' else 0 for label in y_train])
test_labels = torch.tensor([1 if label == 'PII' else 0 for label in y_test])

# Prepare DataLoader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create the DataLoader for our training set.
train_data = TensorDataset(train_input_ids, train_attention_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
test_data = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Training
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup

# Number of training epochs.
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

print('Training on', device)

# Store the average loss after each epoch so we can plot them.
loss_values = []

for epoch_i in range(0, epochs):
    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    t0 = time.time()
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")



    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in test_dataloader:

        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))



# Evaluation on test set
def evaluate(model, dataloader):
    model.eval()

    all_labels = []
    all_preds = []

    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        preds = np.argmax(logits, axis=1)

        all_labels.extend(label_ids)
        all_preds.extend(preds)

    print(classification_report(all_labels, all_preds))

# Evaluation on test set
print("\nEvaluation on test set:")
evaluate(model, test_dataloader)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training on cpu

======== Epoch 1 / 1 ========
Training...

  Average training loss: 0.56
  Training epoch took: 00:00:00

Running Validation...
  Accuracy: 0.88
  Validation took: 00:00:34

Evaluation on test set:
              precision    recall  f1-score   support

           0       0.96      0.75      0.84        36
           1       0.83      0.98      0.90        44

    accuracy                           0.88        80
   macro avg       0.90      0.86      0.87        80
weighted avg       0.89      0.88      0.87        80



In [ ]:

import re
# Function to detect sensitive information using the trained BERT model
def detect_sensitive_info_bert(model, tokenizer, sentence):
    inputs = tokenizer(sentence, return_tensors="pt", max_length=1000, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    #print(predicted_class)
    return "PII" if predicted_class == 1 else "Other"

# Test with sample sentences
sample_sentences = [
    "Customer with Aadhaar number 5828 9552 3855 is requesting a transaction.",
    "Amount:$100 with TxnID:pay_NXDEFGHAS",
    "Pancard: ABCDE1234F belongs to the account holder.",
    "KYC verification successful for customer with Aadhaar number 9876 5432 1098.",
    "Invalid transaction. Please check your details.",
    "Pancard number XYZAB5678G is invalid.",
    "Customer with Aadhaar num 987654321098 is requesting a transaction.",
    "Pancard: XYZAB5678G belongs to the account holder.",
    "Wow My PanCard Number is Amazing and AadhaarCard Number is also Amazing",
    "A refund of Aadhaar - 763285734061 with has been initiated for your recent transaction.",
    "This is a sample sentence that doesn't contain anything",
    "Thank you for paying with DQEPP8752A",
    "Thank you for paying with Pancard:DQEPP8752A",
    '{"PanCard":"XYZAB5678G", "message":"This is ID card"}',
    '{"aadhaarNumber":"694361360105", "message":"This is ID card"}'
]

def extract_custom_entities(text):
    # Define regular expressions for Aadhaar card and Pan card patterns
    aadhaar_pattern = r'\b\d{4}\s?\d{4}\s?\d{4}\b'  # Aadhaar card pattern
    pancard_pattern = r'[A-Z]{5}\d{4}[A-Z]'         # Pan card pattern

    # Search for Aadhaar card numbers and Pan card numbers in the text
    aadhaar_matches = re.findall(aadhaar_pattern, text)
    pancard_matches = re.findall(pancard_pattern, text)

    # Combine the matches and their labels
    custom_entities = [("Aadhaar", match) for match in aadhaar_matches]
    custom_entities += [("PanCard", match) for match in pancard_matches]

    return custom_entities

def redact_custom_entities(text):
    # Define regular expressions for Aadhaar card and Pan card patterns
    aadhaar_pattern = r'\b\d{4}\s?\d{4}\s?\d{4}\b'  # Aadhaar card pattern
    pancard_pattern = r'[A-Z]{5}\d{4}[A-Z]'         # Pan card pattern

    # Redact Aadhaar card numbers
    redacted_text = re.sub(aadhaar_pattern, "[REDACTED]", text)

    # Redact Pan card numbers
    redacted_text = re.sub(pancard_pattern, "[REDACTED]", redacted_text)

    return redacted_text

print("\nDetected labels for sample sentences:\n")
for sentence in sample_sentences:
    label = detect_sensitive_info_bert(model, tokenizer, sentence)
    print(f"\nSentence: '{sentence}'\nLabel : {label}")
    if label == 'PII':
      entities = extract_custom_entities(sentence)
      print("Custom entities:", entities)
      redacted_text = redact_custom_entities(sentence)
      print("Redacted text:", redacted_text)


Detected labels for sample sentences:


Sentence: 'Customer with Aadhaar number 5828 9552 3855 is requesting a transaction.'
Label : PII
Custom entities: [('Aadhaar', '5828 9552 3855')]
Redacted text: Customer with Aadhaar number [REDACTED] is requesting a transaction.

Sentence: 'Amount:$100 with TxnID:pay_NXDEFGHAS'
Label : Other

Sentence: 'Pancard: ABCDE1234F belongs to the account holder.'
Label : PII
Custom entities: [('PanCard', 'ABCDE1234F')]
Redacted text: Pancard: [REDACTED] belongs to the account holder.

Sentence: 'KYC verification successful for customer with Aadhaar number 9876 5432 1098.'
Label : PII
Custom entities: [('Aadhaar', '9876 5432 1098')]
Redacted text: KYC verification successful for customer with Aadhaar number [REDACTED].

Sentence: 'Invalid transaction. Please check your details.'
Label : PII
Custom entities: []
Redacted text: Invalid transaction. Please check your details.

Sentence: 'Pancard number XYZAB5678G is invalid.'
Label : PII
Custom entities: [('

In [ ]:
def write_to_log(data_list, filename):
    with open(filename, 'w') as file:
        for item in data_list:
            file.write(str(item) + '\n')

def read_from_log(filename):
    data_list = []
    with open(filename, 'r') as file:
        for line in file:
            data_list.append(line.strip())
    return data_list

train_labels, train_text, train_PII = fake_.create_pii_text_train(n_text = 100)

write_to_log(train_text, 'application.log')
datalist = read_from_log('application.log')
list_redacted=[]
for sentence in datalist:
    label = detect_sensitive_info_bert(model, tokenizer, sentence)
    #print(f"\nSentence: '{sentence}'\nLabel : {label}")
    if label == 'PII':
      entities = extract_custom_entities(sentence)
      redacted_text = redact_custom_entities(sentence)
      list_redacted.append(redacted_text)
write_to_log(list_redacted, 'application_redacted.log')
print('Completed')


100%|██████████| 300/300 [00:00<00:00, 16327.24it/s]


Completed


In [ ]:
torch.save(model.state_dict(), 'binary_model.pth')

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import re
# Function to detect sensitive information using the trained BERT model
def detect_sensitive_info_bert(model, tokenizer, sentence):
    inputs = tokenizer(sentence, return_tensors="pt", max_length=1000, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    #print(predicted_class)
    return "PII" if predicted_class == 1 else "Other"



def extract_custom_entities(text):
    # Define regular expressions for Aadhaar card and Pan card patterns
    aadhaar_pattern = r'\b\d{4}\s?\d{4}\s?\d{4}\b'  # Aadhaar card pattern
    pancard_pattern = r'[A-Z]{5}\d{4}[A-Z]'         # Pan card pattern

    # Search for Aadhaar card numbers and Pan card numbers in the text
    aadhaar_matches = re.findall(aadhaar_pattern, text)
    pancard_matches = re.findall(pancard_pattern, text)

    # Combine the matches and their labels
    custom_entities = [("Aadhaar", match) for match in aadhaar_matches]
    custom_entities += [("PanCard", match) for match in pancard_matches]

    return custom_entities

def redact_custom_entities(text):
    # Define regular expressions for Aadhaar card and Pan card patterns
    aadhaar_pattern = r'\b\d{4}\s?\d{4}\s?\d{4}\b'  # Aadhaar card pattern
    pancard_pattern = r'[A-Z]{5}\d{4}[A-Z]'         # Pan card pattern

    # Redact Aadhaar card numbers
    redacted_text = re.sub(aadhaar_pattern, "[REDACTED]", text)

    # Redact Pan card numbers
    redacted_text = re.sub(pancard_pattern, "[REDACTED]", redacted_text)

    return redacted_text
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Load the saved state dictionary
state_dict = torch.load('binary_model.pth')

# Initialize the model with the saved state dictionary
model.load_state_dict(state_dict)
label=''
# Define the Streamlit application
def main():
    st.title("Strengthening Log Security: PII & Sensitive Data Identification and Masking - Demo")
    # Add an input textarea for text input
    input_text = st.text_area("Enter Log Text:")
    label = detect_sensitive_info_bert(model, tokenizer, input_text)
    st.write(f"\nSentence: '{input_text}'\nLabel : {label}")
    if label == 'PII':
      entities = extract_custom_entities(input_text)
      st.write("Custom entities:", entities)
      redacted_text = redact_custom_entities(input_text)
      st.write("Redacted text:", redacted_text)

# Run the Streamlit app
if __name__ == '__main__':
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py & > logs.txt & npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

34.141.159.227


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.141.159.227:8501

npx: installed 22 in 3.118s
your url is: https://green-things-yawn.loca.lt
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
Y